# Visualizing the knowledge graph with `yfiles-jupyter-graphs`

This notebook is a partial copy of [local_search.ipynb](../../local_search.ipynb) that shows how to use `yfiles-jupyter-graphs` to add interactive graph visualizations of the parquet files  and how to visualize the result context of `graphrag` queries (see at the end of this notebook).

In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [ ]:
INPUT_DIR = "output/20240828-115326/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")[:15000]
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")[:15000]
print(len(entity_df))
print(len(entity_embedding_df))

15000
15000


#### Read relationships

In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")[:15000]
relationships = read_indexer_relationships(relationship_df)
print(len(relationship_df))
print(len(relationships))

15000
15000


# Visualizing nodes and relationships with `yfiles-jupyter-graphs`

`yfiles-jupyter-graphs` is a graph visualization extension that provides interactive and customizable visualizations for structured node and relationship data.

In this case, we use it to provide an interactive visualization for the knowledge graph of the [local_search.ipynb](../../local_search.ipynb) sample by passing node and relationship lists converted from the given parquet files. The requirements for the input data is an `id` attribute for the nodes and `start`/`end` properties for the relationships that correspond to the node ids. Additional attributes can be added in the `properties` of each node/relationship dict:

In [5]:
# %pip install yfiles_jupyter_graphs --quiet
from yfiles_jupyter_graphs import GraphWidget


# converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_entities_to_dicts(df):
    """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
    nodes_dict = {}
    for _, row in df.iterrows():
        # Create a dictionary for each row and collect unique nodes
        node_id = row["title"]
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
    return list(nodes_dict.values())


# converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
    relationships = []
    for _, row in df.iterrows():
        # Create a dictionary for each row
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships


w = GraphWidget()
w.directed = True
w.nodes = convert_entities_to_dicts(entity_df)
w.edges = convert_relationships_to_dicts(relationship_df)

## Configure data-driven visualization

The additional properties can be used to configure the visualization for different use cases.

In [6]:
# show title on the node
w.node_label_mapping = "title"


# map community to a color
def community_to_color(community):
    """Map a community to a color."""
    colors = [
        "crimson",
        "darkorange",
        "indigo",
        "cornflowerblue",
        "cyan",
        "teal",
        "green",
    ]
    return (
        colors[int(community) % len(colors)] if community is not None else "lightgray"
    )


def edge_to_source_community(edge):
    """Get the community of the source node of an edge."""
    source_node = next(
        (entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community"]
    return source_node_community if source_node_community is not None else None


w.node_color_mapping = lambda node: community_to_color(node["properties"]["community"])
w.edge_color_mapping = lambda edge: community_to_color(edge_to_source_community(edge))
# map size data to a reasonable factor
w.node_scale_factor_mapping = lambda node: 0.5 + node["properties"]["size"] * 1.5 / 20
# use weight for edge thickness
w.edge_thickness_factor_mapping = "weight"

## Automatic layouts

The widget provides different automatic layouts that serve different purposes: `Circular`, `Hierarchic`, `Organic (interactiv or static)`, `Orthogonal`, `Radial`, `Tree`, `Geo-spatial`.

For the knowledge graph, this sample uses the `Circular` layout, though `Hierarchic` or `Organic` are also suitable choices.

In [7]:
# Use the circular layout for this visualization. For larger graphs, the default organic layout is often preferrable.
w.circular_layout()

## Display the graph

In [8]:
display(w)
image_path = "graph_output.png"
w.save_as_image(image_path)

GraphWidget(layout=Layout(height='800px', width='100%'))

# Visualizing the result context of `graphrag` queries

The result context of `graphrag` queries allow to inspect the context graph of the request. This data can similarly be visualized as graph with `yfiles-jupyter-graphs`.

## Making the request

The following cell recreates the sample queries from [local_search.ipynb](../../local_search.ipynb).

In [21]:
# setup (see also ../../local_search.ipynb)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")
# claims = read_indexer_covariates(covariate_df)
# covariates = {"claims": claims}
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

# api_key = os.environ["GRAPHRAG_API_KEY"]
# llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
# embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

api_key = "sk-urFGAQRThR6pysea0aC93bD27fA34bA69811A9254aAaD8B2"
llm_model = "gpt-4o-mini"
embedding_model = "text-embedding-3-small"
api_base = "https://vip.apiyi.com/v1"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_base=api_base,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=api_base,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

# context_builder = LocalSearchMixedContext(
#     community_reports=reports,
#     text_units=text_units,
#     entities=entities,
#     relationships=relationships,
#     covariates=covariates,
#     entity_text_embeddings=description_embedding_store,
#     embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
#     text_embedder=text_embedder,
#     token_encoder=token_encoder,
# )

context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 4_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

## Run local search on sample queries

In [23]:
query = """
你是一个高级材料科学家。你的任务是根据用户提供的信息和从Outer_Retrieval_Admin检索的知识片段，详细回答用户关于材料合成相关的问题（至少1000字）。
用户问题中通常不仅仅只报错合成方案的要求，你需要根据情况自行判断。

(1)当用户问题中存在关于材料合成的要求时，你需要：
利用你的专业知识来仔细识别用户需求，并选择最合适的方法和物质，确保目标材料的准确合成。你的合成方案应分为以下几个部分：

1. **合成条件（Synthesis Conditions）**：说明合成最终材料所需的环境或操作条件，如温度、压力、pH值、溶剂等。
2. **材料及量（Materials & Amounts Required）**：列出合成最终产品所需的初始物质及其具体数量，包括任何催化剂或溶剂。确保所有溶液以毫升（mL）为单位，所有固体以毫克（mg）为单位。
3. **设备及规格（Equipment & Specifications）**：详细列出合成所需的装置及其技术规格（如容量、温度控制范围），包括任何特殊的反应器、搅拌器或测量工具。
4. **合成序列（Synthesis Sequence）**：阐明前驱体和最终材料的合成顺序，描述每一步骤所需的材料数量、设备尺寸和操作程序（如混合、加热、冷却等）。
5. **最终材料的逐步合成要求（Step-by-Step Requirements for Final Material Synthesis）**：将合成步骤分解为若干子步骤，并具体说明每一子步骤中涉及的试剂数量、设备大小（如实验室规模或工业级），以及具体的操作过程。
6. **合成材料的表征（Characterization of Synthesized Material）**：说明用于分析和确认所合成材料结构、纯度或其他性质的方法，这些方法可能包括光谱学、色谱学或显微技术。
7. **其他注意事项（Additional Considerations）**：强调其他相关因素，如安全措施、可扩展性挑战、存储要求或环境影响。

创建一个全面的实验方案，你的目标是生产出一个准确、详尽且可在实际实验室中执行的合成计划。

(2)当用户问题中存在除材料合成要求外的其他的要求时，你需要尽可能详细全面组织答案进行回答。

How to Synthesis CsPbBr3 nanocubes at room temperature?
"""

result = await search_engine.asearch(query)
print(result.response)

# Synthesis of CsPbBr3 Nanocubes at Room Temperature

The synthesis of cesium lead bromide (CsPbBr3) nanocubes at room temperature is a significant area of research in materials science, particularly for applications in optoelectronics and photovoltaics. This guide outlines a comprehensive synthesis plan, detailing the necessary conditions, materials, equipment, synthesis sequence, characterization methods, and additional considerations.

## 1. Synthesis Conditions

The synthesis of CsPbBr3 nanocubes at room temperature requires specific conditions to ensure the formation of high-quality nanocrystals. The key conditions include:

- **Temperature**: Room temperature (approximately 20-25°C).
- **Atmosphere**: Inert atmosphere (nitrogen or argon) to prevent moisture and oxygen interference.
- **pH**: Neutral pH is preferred to maintain the stability of the precursor solutions.
- **Solvent**: Use of a suitable solvent such as dimethylformamide (DMF) or dimethyl sulfoxide (DMSO) to dissolve

In [ ]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

## Inspecting the context data used to generate the response

In [24]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,5314,SYNTHESIS PROCESS,"The ""SYNTHESIS PROCESS"" encompasses a variety ...",6,True
1,2947,SYNTHESIS,Synthesis refers to the comprehensive process ...,62,True
2,7101,SYNTHESIS METHOD,"The ""SYNTHESIS METHOD"" encompasses a variety o...",10,True
3,7367,SYNTHESIS METHODS,"The entity ""SYNTHESIS METHODS"" encompasses a w...",28,True
4,2477,MATERIAL,"The entity ""MATERIAL"" encompasses a diverse ra...",47,True


In [25]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,13017,XRD,CSPB3 NCS,XRD (X-ray Diffraction) is a critical analytic...,5.0,1409,2,True
1,13523,XRD,CS3CU2I5 NCS,XRD is used to confirm the crystal phase of Cs...,1.0,1306,2,True
2,2333,EXCITON BINDING ENERGY,CSPB3 NCS,The exciton binding energy of CsPbBr3 NCs is a...,1.0,299,2,True
3,2267,EXCITON BINDING ENERGY,CS3CU2I5 NCS,The exciton binding energy of Cs3Cu2I5 NCs is ...,1.0,196,2,True
4,9685,PEROVSKITE,PRECURSOR SOLUTION,The precursor solution is essential for the fa...,1.0,2432,1,True


## Visualizing the result context as graph

In [26]:
"""
Helper function to visualize the result context with `yfiles-jupyter-graphs`.

The dataframes are converted into supported nodes and relationships lists and then passed to yfiles-jupyter-graphs.
Additionally, some values are mapped to visualization properties.
"""


def show_graph(result):
    """Visualize the result context with yfiles-jupyter-graphs."""
    from yfiles_jupyter_graphs import GraphWidget

    if (
        "entities" not in result.context_data
        or "relationships" not in result.context_data
    ):
        msg = "The passed results do not contain 'entities' or 'relationships'"
        raise ValueError(msg)

    # converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_entities_to_dicts(df):
        """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
        nodes_dict = {}
        for _, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["entity"]
            if node_id not in nodes_dict:
                nodes_dict[node_id] = {
                    "id": node_id,
                    "properties": row.to_dict(),
                }
        return list(nodes_dict.values())

    # converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_relationships_to_dicts(df):
        """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
        relationships = []
        for _, row in df.iterrows():
            # Create a dictionary for each row
            relationships.append({
                "start": row["source"],
                "end": row["target"],
                "properties": row.to_dict(),
            })
        return relationships

    w = GraphWidget()
    # use the converted data to visualize the graph
    w.nodes = convert_entities_to_dicts(result.context_data["entities"])
    w.edges = convert_relationships_to_dicts(result.context_data["relationships"])
    w.directed = True
    # show title on the node
    w.node_label_mapping = "entity"
    # use weight for edge thickness
    w.edge_thickness_factor_mapping = "weight"
    display(w)


show_graph(result)

GraphWidget(layout=Layout(height='700px', width='100%'))